In [1]:
# import pyrealsense2 as rs
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import h5py
import os
import argparse
import multiprocessing as mp
import yaml
import warnings
from queue import LifoQueue
from threading import Thread

In [ ]:
def append_to_hdf5(f, name, value, axis=0):
    f[name].resize(f[name].shape[axis]+1, axis=axis)
    f[name][-1]=value
    
class Device:
    def __init__(self, serial,config, start_t, save=False,savedir=None,experiment=None, name=None,
        save_format='hdf5',preview=False,verbose=False,options=None):
        # print('Initializing %s' %name)
        self.config = config
        self.serial = serial
        self.start_t = start_t
        self.save= save
        self.savedir = savedir
        self.experiment = experiment
        self.name = name
        self.save_format=save_format
        self.preview=preview
        self.verbose = verbose
        self.options = options
        self.started = False
        # print('Done.')

    def start(self, sync_mode='slave'):
        pipeline = rs.pipeline()
        self.config.enable_device(self.serial)
        try:
            pipeline_profile = pipeline.start(self.config)
        except RuntimeError:
            print('Pipeline for camera %s already running, restarting...' %self.serial)
            pipeline.stop()
            time.sleep(1)
            pipeline_profile = pipeline.start(self.config)
        CAPACITY = 10
        # print(dir(pipeline))
        # self.framequeue = rs.framequeue(CAPACITY)
        self.pipeline = pipeline
        self.prof = pipeline_profile
        time.sleep(1)
        self.update_settings(sync_mode)
        if self.save:
            self.initialize_saving()
            print('saving initialized: %s' %self.name)
        if self.preview:
            self.initialize_preview()
        self.started= True

    def initialize_preview(self):
        cv2.namedWindow(self.name, cv2.WINDOW_AUTOSIZE)
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.latest_frame = None
        self.preview_queue = LifoQueue(maxsize=5)
        self.preview_thread = Thread(target=self.preview_worker, args=(self.preview_queue,))
        self.preview_thread.daemon = True
        self.preview_thread.start()

    def preview_worker(self, queue):
        while True:
            item = queue.get()
            # print(item)
            if item is None:
                # print('Stop signal received')
                break
            left, right, count = item
            out = np.vstack((left,right))
            h, w = out.shape
            if self.save:
                out = cv2.resize(out, (w//2,h//2),cv2.INTER_NEAREST)
                out_height = h//2
            else:
                out = cv2.resize(out, (w//3*2,h//3*2),cv2.INTER_NEAREST)
                out_height = h//3*2
            out = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
                    
            # string = '%.4f' %(time_acq*1000)
            string = '%s:%07d' %(self.name, count)
            cv2.putText(out,string,(10,out_height-20), self.font, 0.5,(0,0,255),2,cv2.LINE_AA)
            self.latest_frame = out

            queue.task_done()

    def initialize_saving(self):
        subdir = os.path.join(self.savedir, self.experiment)
        if not os.path.isdir(subdir):
            os.makedirs(subdir)

        fname = self.name + '.h5'
        f = h5py.File(os.path.join(subdir, fname), 'w')
        dset = f.create_dataset('framecount',(0,),maxshape=(None,),dtype=np.int32)
        dset = f.create_dataset('timestamp',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('arrival_time',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('sestime',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('cputime',(0,),maxshape=(None,),dtype=np.float64)

        if self.save_format == 'hdf5':
            # fname = '%s_%s.h5' %(self.experiment, self.name)
            datatype = h5py.special_dtype(vlen=np.dtype('uint8'))
            dset = f.create_dataset('left', (0,), maxshape=(None,),dtype=datatype)
            dset = f.create_dataset('right', (0,), maxshape=(None,),dtype=datatype)

            # self.metadataobj = f
            self.write_frames = self.write_frames_hdf5
        elif self.save_format == 'opencv':
            movname = self.name + '_%04d.bmp'
            # movname = self.name + '.mov'
            # fourcc = cv2.VideoWriter_fourcc(*'raw ')
            # fourcc = 
            self.videoobj = cv2.VideoWriter(os.path.join(subdir, movname), fourcc=0, fps=0, 
                frameSize=(1280,480))
            self.write_frames = self.write_frames_opencv
        else:
            raise NotImplementedError
        self.fileobj = f
        print(subdir)

    def write_frames_opencv(self, left, right, framecount, timestamp,
        arrival_time, sestime, cputime):
        if not hasattr(self, 'fileobj'):
            raise ValueError('Writing for camera %s not initialized.' %self.camname)
        # out = 
        # out = np.hstack((left, right))
        self.videoobj.write(cv2.cvtColor(np.hstack((left, right)), cv2.COLOR_GRAY2BGR))
        append_to_hdf5(self.fileobj,'framecount', framecount)
        append_to_hdf5(self.fileobj,'timestamp', timestamp)
        append_to_hdf5(self.fileobj,'arrival_time', arrival_time)
        append_to_hdf5(self.fileobj,'sestime', sestime)
        append_to_hdf5(self.fileobj, 'cputime', cputime)

    def write_frames_hdf5(self,left,right, framecount, timestamp,
                        arrival_time,sestime,cputime):
        if not hasattr(self, 'fileobj'):
            raise ValueError('Writing for camera %s not initialized.' %self.camname)
        # ret1, ret2 = True, True
        ret1, left_jpg = cv2.imencode('.jpg', left, (cv2.IMWRITE_JPEG_QUALITY,80))
        ret2, right_jpg = cv2.imencode('.jpg', right, (cv2.IMWRITE_JPEG_QUALITY,80))
        if ret1 and ret2:
            append_to_hdf5(self.fileobj, 'left', left_jpg.squeeze())
            append_to_hdf5(self.fileobj, 'right', right_jpg.squeeze())
            append_to_hdf5(self.fileobj,'framecount', framecount)
            append_to_hdf5(self.fileobj,'timestamp', timestamp)
            append_to_hdf5(self.fileobj,'arrival_time', arrival_time)
            append_to_hdf5(self.fileobj,'sestime', sestime)
            append_to_hdf5(self.fileobj, 'cputime', cputime)

    def update_settings(self, sync_mode='master'):
        # sensor = self.prof.get_device().first_depth_sensor()
        # print(dir(sensor))
        # sensor.set_option(rs.option.emitter_enabled,1)
        if sync_mode=='master':
            mode = 1
        elif sync_mode == 'slave':
            mode = 2
        if self.verbose:
            print('%s: %s,%d' %(self.name, sync_mode, mode))
        
        this_device = self.prof.get_device()
        ir_sensors = this_device.query_sensors()[0] # 1 for RGB
        if self.options=='default' or self.options=='large':
            ir_sensors.set_option(rs.option.emitter_enabled,1)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            laser_pwr = ir_sensors.get_option(rs.option.laser_power)
            if self.verbose:
                print("laser power = ", laser_pwr)
            laser_range = ir_sensors.get_option_range(rs.option.laser_power)
            if self.verbose:
                print("laser power range = " , laser_range.min , "~", laser_range.max)
            ir_sensors.set_option(rs.option.laser_power,300)
            ir_sensors.set_option(rs.option.exposure,650)
            ir_sensors.set_option(rs.option.gain,16)

        elif self.options =='calib':
            ir_sensors.set_option(rs.option.emitter_enabled,0)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            ir_sensors.set_option(rs.option.exposure,1200)
            ir_sensors.set_option(rs.option.gain,16)
            self.jpg_quality = 99

        if self.options=='large':
            ir_sensors.set_option(rs.option.exposure,750)
            ir_sensors.set_option(rs.option.laser_power,200)
            ir_sensors.set_option(rs.option.gain,16)
        if self.options=='brighter':
            gain_range = ir_sensors.get_option_range(rs.option.gain)
            if self.verbose:
                print("gain range = " , gain_range.min , "~", gain_range.max)
            ir_sensors.set_option(rs.option.exposure, 500)
            ir_sensors.set_option(rs.option.gain,16)
        # set this to 2 for slave mode, 1 for master!
        ir_sensors.set_option(rs.option.inter_cam_sync_mode, mode)
        # print(ir_sensors.supports(rs.option.inter_cam_sync_mode))
        # this_device.set_option(rs.option.inter_cam_sync_mode,2)
        # from github
        # ir_sensors.set_option(rs.option.frames_queue_size,7)
        # print(ir_sensors.get_option(rs.option.inter_cam_sync_mode))

    def stop_streaming(self):
        # print(dir(self.pipeline))
        try:
            self.pipeline.stop()
            self.config.disable_all_streams()
        except BaseException as e:
            if self.verbose:
                print('Probably tried to call stop before a start.')
                print(e)
            else:
                pass
        # print('%s stopped streaming' %self.name)
        # if self.preview:

    def stop(self):
        # if self.preview:
        if not self.started:
            return
            
        if self.preview:
            self.preview_queue.put(None)
            self.preview_thread.join()
            cv2.destroyWindow(self.name)
        if hasattr(self, 'pipeline'):
            # print('stream')
            self.stop_streaming()
        if hasattr(self, 'videoobj'):
            # print('videoobj')
            self.videoobj.release()
            # del(self.videoobj)
        if hasattr(self, 'fileobj'):
            # print('fileobj')
            self.fileobj.close()
        self.started = False
        print('Cam %s stopped' %self.name) 

    def __del__(self):
        try:
            self.stop()
        except BaseException as e:
            if self.verbose:
                print('Error in destructor of cam %s' %self.name)
                print(e)
            else:
                pass

In [ ]:
class Device:
    def __init__(self, serial,config, start_t, save=False,savedir=None,experiment=None, name=None,
        save_format='hdf5',preview=False,verbose=False,options=None):
        # print('Initializing %s' %name)
        self.config = config
        self.serial = serial
        self.start_t = start_t
        self.save= save
        self.savedir = savedir
        self.experiment = experiment
        self.name = name
        self.save_format=save_format
        self.preview=preview
        self.verbose = verbose
        self.options = options
        self.started = False
        # print('Done.')

    def start(self, sync_mode='slave'):
        pipeline = rs.pipeline()
        self.config.enable_device(self.serial)
        try:
            pipeline_profile = pipeline.start(self.config)
        except RuntimeError:
            print('Pipeline for camera %s already running, restarting...' %self.serial)
            pipeline.stop()
            time.sleep(1)
            pipeline_profile = pipeline.start(self.config)
        CAPACITY = 10
        # print(dir(pipeline))
        # self.framequeue = rs.framequeue(CAPACITY)
        self.pipeline = pipeline
        self.prof = pipeline_profile
        time.sleep(1)
        self.update_settings(sync_mode)
        if self.save:
            self.initialize_saving()
            print('saving initialized: %s' %self.name)
        if self.preview:
            self.initialize_preview()
        self.started= True

    def initialize_preview(self):
        cv2.namedWindow(self.name, cv2.WINDOW_AUTOSIZE)
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.latest_frame = None
        self.preview_queue = LifoQueue(maxsize=5)
        self.preview_thread = Thread(target=self.preview_worker, args=(self.preview_queue,))
        self.preview_thread.daemon = True
        self.preview_thread.start()

    def preview_worker(self, queue):
        while True:
            item = queue.get()
            # print(item)
            if item is None:
                # print('Stop signal received')
                break
            left, right, count = item
            out = np.vstack((left,right))
            h, w = out.shape
            if self.save:
                out = cv2.resize(out, (w//2,h//2),cv2.INTER_NEAREST)
                out_height = h//2
            else:
                out = cv2.resize(out, (w//3*2,h//3*2),cv2.INTER_NEAREST)
                out_height = h//3*2
            out = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
                    
            # string = '%.4f' %(time_acq*1000)
            string = '%s:%07d' %(self.name, count)
            cv2.putText(out,string,(10,out_height-20), self.font, 0.5,(0,0,255),2,cv2.LINE_AA)
            self.latest_frame = out

            queue.task_done()

    def initialize_saving(self):
        subdir = os.path.join(self.savedir, self.experiment)
        if not os.path.isdir(subdir):
            os.makedirs(subdir)

        fname = self.name + '.h5'
        f = h5py.File(os.path.join(subdir, fname), 'w')
        dset = f.create_dataset('framecount',(0,),maxshape=(None,),dtype=np.int32)
        dset = f.create_dataset('timestamp',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('arrival_time',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('sestime',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('cputime',(0,),maxshape=(None,),dtype=np.float64)

        if self.save_format == 'hdf5':
            # fname = '%s_%s.h5' %(self.experiment, self.name)
            datatype = h5py.special_dtype(vlen=np.dtype('uint8'))
            dset = f.create_dataset('left', (0,), maxshape=(None,),dtype=datatype)
            dset = f.create_dataset('right', (0,), maxshape=(None,),dtype=datatype)

            # self.metadataobj = f
            self.write_frames = self.write_frames_hdf5
        elif self.save_format == 'opencv':
            movname = self.name + '_%04d.bmp'
            # movname = self.name + '.mov'
            # fourcc = cv2.VideoWriter_fourcc(*'raw ')
            # fourcc = 
            self.videoobj = cv2.VideoWriter(os.path.join(subdir, movname), fourcc=0, fps=0, 
                frameSize=(1280,480))
            self.write_frames = self.write_frames_opencv
        else:
            raise NotImplementedError
        self.fileobj = f
        print(subdir)

    def write_frames_opencv(self, left, right, framecount, timestamp,
        arrival_time, sestime, cputime):
        if not hasattr(self, 'fileobj'):
            raise ValueError('Writing for camera %s not initialized.' %self.camname)
        # out = 
        # out = np.hstack((left, right))
        self.videoobj.write(cv2.cvtColor(np.hstack((left, right)), cv2.COLOR_GRAY2BGR))
        append_to_hdf5(self.fileobj,'framecount', framecount)
        append_to_hdf5(self.fileobj,'timestamp', timestamp)
        append_to_hdf5(self.fileobj,'arrival_time', arrival_time)
        append_to_hdf5(self.fileobj,'sestime', sestime)
        append_to_hdf5(self.fileobj, 'cputime', cputime)

    def write_frames_hdf5(self,left,right, framecount, timestamp,
                        arrival_time,sestime,cputime):
        if not hasattr(self, 'fileobj'):
            raise ValueError('Writing for camera %s not initialized.' %self.camname)
        # ret1, ret2 = True, True
        ret1, left_jpg = cv2.imencode('.jpg', left, (cv2.IMWRITE_JPEG_QUALITY,80))
        ret2, right_jpg = cv2.imencode('.jpg', right, (cv2.IMWRITE_JPEG_QUALITY,80))
        if ret1 and ret2:
            append_to_hdf5(self.fileobj, 'left', left_jpg.squeeze())
            append_to_hdf5(self.fileobj, 'right', right_jpg.squeeze())
            append_to_hdf5(self.fileobj,'framecount', framecount)
            append_to_hdf5(self.fileobj,'timestamp', timestamp)
            append_to_hdf5(self.fileobj,'arrival_time', arrival_time)
            append_to_hdf5(self.fileobj,'sestime', sestime)
            append_to_hdf5(self.fileobj, 'cputime', cputime)

    def update_settings(self, sync_mode='master'):
        # sensor = self.prof.get_device().first_depth_sensor()
        # print(dir(sensor))
        # sensor.set_option(rs.option.emitter_enabled,1)
        if sync_mode=='master':
            mode = 1
        elif sync_mode == 'slave':
            mode = 2
        if self.verbose:
            print('%s: %s,%d' %(self.name, sync_mode, mode))
        
        this_device = self.prof.get_device()
        ir_sensors = this_device.query_sensors()[0] # 1 for RGB
        if self.options=='default' or self.options=='large':
            ir_sensors.set_option(rs.option.emitter_enabled,1)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            laser_pwr = ir_sensors.get_option(rs.option.laser_power)
            if self.verbose:
                print("laser power = ", laser_pwr)
            laser_range = ir_sensors.get_option_range(rs.option.laser_power)
            if self.verbose:
                print("laser power range = " , laser_range.min , "~", laser_range.max)
            ir_sensors.set_option(rs.option.laser_power,300)
            ir_sensors.set_option(rs.option.exposure,650)
            ir_sensors.set_option(rs.option.gain,16)

        elif self.options =='calib':
            ir_sensors.set_option(rs.option.emitter_enabled,0)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            ir_sensors.set_option(rs.option.exposure,1200)
            ir_sensors.set_option(rs.option.gain,16)
            self.jpg_quality = 99

        if self.options=='large':
            ir_sensors.set_option(rs.option.exposure,750)
            ir_sensors.set_option(rs.option.laser_power,200)
            ir_sensors.set_option(rs.option.gain,16)
        if self.options=='brighter':
            gain_range = ir_sensors.get_option_range(rs.option.gain)
            if self.verbose:
                print("gain range = " , gain_range.min , "~", gain_range.max)
            ir_sensors.set_option(rs.option.exposure, 500)
            ir_sensors.set_option(rs.option.gain,16)
        # set this to 2 for slave mode, 1 for master!
        ir_sensors.set_option(rs.option.inter_cam_sync_mode, mode)
        # print(ir_sensors.supports(rs.option.inter_cam_sync_mode))
        # this_device.set_option(rs.option.inter_cam_sync_mode,2)
        # from github
        # ir_sensors.set_option(rs.option.frames_queue_size,7)
        # print(ir_sensors.get_option(rs.option.inter_cam_sync_mode))

    def stop_streaming(self):
        # print(dir(self.pipeline))
        try:
            self.pipeline.stop()
            self.config.disable_all_streams()
        except BaseException as e:
            if self.verbose:
                print('Probably tried to call stop before a start.')
                print(e)
            else:
                pass
        # print('%s stopped streaming' %self.name)
        # if self.preview:

    def stop(self):
        # if self.preview:
        if not self.started:
            return
            
        if self.preview:
            self.preview_queue.put(None)
            self.preview_thread.join()
            cv2.destroyWindow(self.name)
        if hasattr(self, 'pipeline'):
            # print('stream')
            self.stop_streaming()
        if hasattr(self, 'videoobj'):
            # print('videoobj')
            self.videoobj.release()
            # del(self.videoobj)
        if hasattr(self, 'fileobj'):
            # print('fileobj')
            self.fileobj.close()
        self.started = False
        print('Cam %s stopped' %self.name) 

    def __del__(self):
        try:
            self.stop()
        except BaseException as e:
            if self.verbose:
                print('Error in destructor of cam %s' %self.name)
                print(e)
            else:
                pass